In [ ]:
-------------------------------------------------------------------------------------------------------------------------------
!!!!!!!!!!!!!!!!!!!!! 'https://www.accordbox.com/blog/how-crawl-infinite-scrolling-pages-using-python/' !!!!!!!!!!!!!!!!!!!!!!!
-------------------------------------------------------------------------------------------------------------------------------

In [54]:
# -----------------------------------------------------------------------------------------------------------------------------
# ---------------------------------------- importer les librairies nécessaires-------------------------------------------------

from selenium.webdriver.support import expected_conditions as EC 
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium import webdriver
from bs4 import BeautifulSoup
from random import randint
from lxml import html
import pandas as pd
import requests
import time  

In [56]:
# -----------------------------------------------------------------------------------------------------------------------------
# -------------------------------------- récupère les liens de chaque page produit --------------------------------------------

url = []


# le client entre la 2e page du produit recherché, pour qu'on puisse en déduire la 'base url'
print("Entrez le produit recheché:")
produit = input().split(' ')        

# Base de l'Url, on le construit à partir du produit (puis par la suite on ajoute le numéro de chaque page)
# si on entre 'iphone 12 pro'                                  --->   cela nous donne:
# https://fr.aliexpress.com/af/iPhone-12-pro.html?trafficChannel=af&d=y&CatId=0&SearchText=iPhone+12+pro&ltype=affiliate&SortType=default&page=4
base_url = ''.join(['https://fr.aliexpress.com/af/', ''.join(['-' + produit[i] for i in range(1,len(produit)) if len(produit) > 1]),
                    '.html?trafficChannel=af&d=y&CatId=0&SearchText=',
                    ''.join(['+' + produit[i] for i in range(1,len(produit)) if len(produit) > 1]), '&ltype=affiliate&SortType=default&page={}']) 

# ouvrir le navigateur Chrome, et aller sur la 1ere page qu'on a quand on recherche un produit
driver = webdriver.Chrome(ChromeDriverManager().install())

try:              # au cas où il y a moins de 100 pages
    for i in range(1,101):  # sur 100 pages max

        # aller à l'url de chaque page disponible: page1, page2, ect 
        driver.get(base_url.format(i))

        # scroller tout en bas pour bien faire apparaitre tous les éléments de la page
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        time.sleep(randint(3,5))

        # récupérer tous les liens de la page                
        elems = driver.find_elements_by_xpath("//a[@href]")
        for elem in elems:
            # ajouter seulement ceux qui permettent d'accéder au fiches produits (/'item')
            if 'https://fr.aliexpress.com/item/' in str(elem.get_attribute("href")):
                url.append(str(elem.get_attribute("href")))
                
        # enlever les liens en doublons
        url = list(set(url))
        
        
        time.sleep(randint(5,12))
except:pass

    
#print(url)
print(len(url))
driver.quit()

Entrez le produit recheché:


 iPhone 12 pro


[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 102.0.5005
[WDM] - Get LATEST chromedriver version for 102.0.5005 google-chrome
[WDM] - Driver [/Users/tam/.wdm/drivers/chromedriver/mac64/102.0.5005.61/chromedriver] found in cache
/var/folders/m2/fdycypmn5fs5tzy134bmr9dm0000gn/T/ipykernel_19432/4280982213.py:19: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())
/var/folders/m2/fdycypmn5fs5tzy134bmr9dm0000gn/T/ipykernel_19432/4280982213.py:33: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  elems = driver.find_elements_by_xpath("//a[@href]")


760


In [57]:
# -----------------------------------------------------------------------------------------------------------------------------
# ---------------------------------------- récupère infos sur chaque page produit ---------------------------------------------

Prix = []
Nom = []


for i in url:
    # recupère html de chaque page                
    r = requests.get(i)  
        # Convertit en chaine de caractères puis en objet BS
    soup = BeautifulSoup(r.content, 'lxml')

    # récupère le titre (qui contient le prix), supprime le début '<meta content=' pour garder que le 1er élément (le prix) de la liste (de split)
    Prix.append(str(soup.find("meta", property="og:title")).replace('<meta content="', '').split(' ')[0])
    Nom.append(soup.find('title').text.replace('| AliExpress','')) # prends le titre du produit (et enlève la fin inutile)


# tableau des données        
tableau = pd.DataFrame(list(zip(Nom,Prix)), columns = ['Produit','Prix (€)'])

# transférer les données dans un fichier CSV
tableau.to_csv(''.join(produit)+'-AliExpress_fichePrix.csv')
tableau     

,Produit,Prix (€)
0,Tokyo Vengeurs Anime Pour iPhone 11 13 Pro Max...,1.9€
1,"Coque de téléphone en TPU souple coloré, motif...",24.14€
2,Étui portefeuille à rabat en cuir PU avec fent...,2.45€
3,GYKZ – coque de téléphone transparente en TPU ...,1.61€
4,Magsafing – coque rigide et transparente pour ...,12.03€
...,...,...
755,Marilyn Monroe – coque souple transparente pou...,3.43€
756,Rappeur Brent Faiyaz pour iPhone 11 12 13 Pro ...,2.72€
757,Coque de téléphone portable avec bracelet de p...,26.16€
758,"Coque de téléphone Apple à motif de poisson, é...",1.7€



#### **------------------------------------------------------  Recherches  ----------------------------------------------------------**


In [ ]:
import requests                       # pour aller chercher l'url
from bs4 import BeautifulSoup         # pour découper le html 
import time                           
import pandas as pd                   # pour classer les données dans un tableau
import csv
from time import sleep
from random import randint
import re                             # regex



 

# le client entre la 2e page du produit recherché, pour qu'on puisse en déduire la 'base url'
print("Entrez le produit recheché:")
produit = input().split(' ')        

# Base de l'Url, on le construit à partir du produit (puis par la suite on ajoute le numéro de chaque page)
# si on entre 'table basse ronde'                                  --->   cela nous donne 'https://www.amazon.fr/s?k=table+basse+ronde&page='  
base_url = ''.join(['https://www.alibaba.com/trade/search?spm=a2700.galleryofferlist.0.0.3bf24ec1NDaDf8&IndexArea=product_en&SearchText=',
                    produit[0], ''.join(['_' + produit[i] for i in range(1,len(produit)) if len(produit) > 1]), '&page={}'])    #{}

# listes contenant le Noms des produits et leurs Prix
Prix = []
Noms_Produits = []
Vendeurs = []


while True: 
    
    # boucle pour assurer la rotation des pages: on incrémente le nombre dans l'url: 'page1', 'page2'...
    for i in range(1,101):

        # recupère html de chaque page                
        r = requests.get(base_url.format(i))  # page=1, page =2... 
        # Convertit en chaine de caractères puis en objet BS
        contentPage = BeautifulSoup(r.content, 'lxml')



        for bloc in contentPage.findAll('div', {'class':'list-no-v2-inner m-gallery-product-item-v2 img-switcher-parent'}):

            #1) - récupère les noms de chaque produit, et les ajoute à la liste 'Noms_Produits' 
            if bloc.find('p', {'class':'elements-title-normal__content large'}): 
                Noms_Produits.append(bloc.find('p', {'class':'elements-title-normal__content large'}).getText())
            else:Noms_Produits.append('none')


            #2) - récupère les prix, et les ajoute à la liste 'Prix'     ---> s'il n y a pas de prix parce que tout vendu, alors on ajoute 'vendus'   
            if bloc.find('span', {'class':'elements-offer-price-normal__promotion'}): 
                Prix.append(bloc.find('span', {'class':'elements-offer-price-normal__promotion'}).getText())
            else:Prix.append('none')

            sleep(randint(1,4))

            #3) - récupère le vendeur, et l'ajoute à la liste 'Vendeurs' (! ---> récupère ça sur la page qu'on a quand on clique sur le produit)                       
            try: 
                lien_vendeurProduit = bloc.find('a',
                        {'class':'elements-title-normal one-line'}, href=True)['href']
    #--------------------------------------------------------------------------------------------------------------------------------------------#
                r = requests.get('https:' + lien_vendeurProduit)  # page=1, page =2...
                # Convertit en chaine de caractères puis en objet BS
                contentPageVendeur = BeautifulSoup(r.content, 'lxml')   
                if 'Toutes nos excuses' in contentPageVendeur.getText()[:21]: print(contentPageVendeur.getText()[:20], '3')
    #--------------------------------------------------------------------------------------------------------------------------------------------#            
    #             # récupérer le nom du vendeur
    #             Vendeurs.append(contentPageVendeur.find('a', {'id':'bylineInfo'}).getText().replace('Visiter la boutique ', ''))
            except:pass
    #               Vendeurs.append('none')
    #         sleep(randint(2,5))



        sleep(15)
        print('Page: '+ str(i) + '/' + '100 or less' + ', fait')        



print(base_url)
# tableau des données        
tableau = pd.DataFrame(list(zip(Noms_Produits,Prix, Vendeurs)), columns = ['produit','prix (€)', 'vendeur'])

# transférer les données dans un fichier CSV
tableau.to_csv(''.join(produit)+'-Aliexpress_fichePrix.csv')
tableau      


In [ ]:
###############################################################################################################################

#                    -          autre méthode qui peut être utile (si besoin de plus d'infos)           -                     #

In [ ]:
###############################################################################################################################

In [ ]:
# -----------------------------------------------------------------------------------------------------------------------------
# ---------------------------------------- récupère infos sur chaque page produit ---------------------------------------------

from requests_html import AsyncHTMLSession
import pyppdf.patch_pyppeteer


Product_name = []
Rating = []
Review_count = []
Price = []



# set headers
header = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.61 Safari/537.36'}


for lien in url:
    try:
        # Start html session and send get request:
        asession = AsyncHTMLSession()
        r = await asession.get(lien)

        # Render Java Script:
        await r.html.arender()


        # Get product name:
        h1 = r.html.find('h1')
        ' '.join(h1[0].text.split(' ')[:10])
        Product_name.append(h1)

        # Product Rating:
        rating = r.html.find('.overview-rating-average')
        ''.join(rating[0].text.split(' ')[0])
        Rating.append(rating)

        # Review Count:
        rating_count = r.html.find('.product-reviewer-reviews')
        ''.join(rating_count[0].text.split(' ')[0])
        Review_count.append(rating_count)

        # Product price:
        price = r.html.find('.product-price-value')
        ''.join(price[0].text.split(' ')[0:2])
        Price.append(price)
    except:pass
    
    # afficher la progression
    


# tableau des données        
tableau = pd.DataFrame(list(zip(Product_name,Rating,Review_count,Price)), columns = ['Produit', 'Note', "Nombre d'avis", 'Prix (€)'])

# transférer les données dans un fichier CSV
tableau.to_csv(''.join(produit)+'-AliExpress_fichePrix.csv')
tableau   